In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
chunk_size = 200
collection_name = f"mycollection_{chunk_size}"
embedding_model = "text-embedding-3-large"

top_k = 4
batch_size = 100

query="高町なのはがユーノから譲り受けたものは何ですか？"

### 埋め込みモデルの準備

In [3]:
from langchain_openai import OpenAIEmbeddings

# 埋め込みモデルの準備
embeddings = OpenAIEmbeddings(model=embedding_model)

### ドキュメントの読み込みとチャンキング

In [4]:
from langchain_community.document_loaders import PDFMinerLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# ドキュメントの読み込み
loader = DirectoryLoader(
    "./data/",
    glob="**/*.pdf",  # PDFファイルを対象に
    show_progress=True,
    loader_cls=PDFMinerLoader  # PDFMinerLoaderを使用
)

In [5]:
import unicodedata

# ドキュメントを Unicode正規化して分割
documents = []
for doc in loader.load():
    # すべての改行を削除（単語の途中でチャンキングされることを防ぐため）
    doc.page_content = unicodedata.normalize('NFKC', doc.page_content.replace("\n", ""))
    chunks = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_size // 10,
        separators=[
            "。",
            "、",
            " ",
            "」",
            "）",
            "｝",
            "〕",
            "》"
        ],
        keep_separator="end"
    ).split_documents([doc])
    documents.extend(chunks)

print(f"合計チャンク数: {len(documents)}")

100%|██████████| 19/19 [00:49<00:00,  2.62s/it]

合計チャンク数: 3375


In [6]:
for i, doc in enumerate(documents[:5]):
    print(f"\n=== Document {i+1} ===")
    print(f"Content: {doc.page_content}")
    print(f"文字数: {len(doc.page_content)}")
    print(f"Source: {doc.metadata['source']}")


=== Document 1 ===
Content: ハリー‧ポッター (架空の人物)出典: フリー百科事典『ウィキペディア(Wikipedia)』ハリー‧ポッターシリーズ > ハリー‧ポッターシリーズの登場人物一覧 > ハリー‧ポッター (架空の人物)ハリー‧ジェームズ‧ポッター(英: Harry JamesPotter)は、J‧K‧ローリングの小説『ハリー‧ポッター』シリーズおよび、その派生作品に登場する架空の人物であり、同シリーズの主人公。
文字数: 198
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 2 ===
Content: ホグワーツ魔法魔術学校グリフィンドール寮の男子生徒となる。孤児として母親の親類の伯母夫婦の家で不遇な暮らしをして育った。11歳を迎える年のある日突然、ホグワーツから入学許可証が届いたのをきっかけに、亡くなった両親が魔法使いであったこと、そして出生時に下された予言により、闇の魔法使いヴォルデモートを倒す宿命を自分が負っていると告げられる。
文字数: 169
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 3 ===
Content: マグル界では一介の少年に過ぎない生活を送っていたが、魔法界では本人が戶惑うほど重要な人物として、あまねく人々から知られている。一人前の魔法使いになるべく、同級生のロン‧ウィーズリーやハーマイオニー‧グレンジャーらとともに、ホグワーツにて学生生活を送りつつ、宿敵のヴォルデモートなどの闇の魔法使いたちによる数々の陰謀に立ち向かう冒険の日々を通して、たくましく成⻑していく姿が物語で描かれている。
文字数: 196
Source: data\Wikipedia-ハリー・ポッター_(架空の人物).pdf

=== Document 4 ===
Content: 人物名前‧外見魔 法 界 で は 「 生 き 残 っ た 男 の 子   (The  boy  wholived) 」と呼ばれる。髪の毛は黒い癖毛で、瞳は明るい緑色。小顔で細面で、近視のため丸眼鏡を着用。同年代に比べ小柄で痩せているが、第6巻『謎のプリンス』では前巻と比べて身⻑がかなり伸びたとされている。
文字数: 

### VectorStoreの準備

In [7]:
import chromadb
from langchain_chroma import Chroma


# Chromaクライアントを初期化
try:
    persistent_client = chromadb.PersistentClient(path="./chroma_db")
    # 既存のChromaコレクションを削除
    try:
        persistent_client.delete_collection(collection_name)
        print(f"既存のコレクション '{collection_name}' を削除しました")
    except ValueError as e:
        print(f"既存のコレクション '{collection_name}' は存在しませんでした")
    except Exception as e:
        print(f"コレクションの削除中にエラーが発生しました: {str(e)}")

    vector_store = Chroma(
        client=persistent_client,
        collection_name=collection_name,
        embedding_function=embeddings,
        collection_metadata={"hnsw:space": "ip"} 
    )
    print(f"コレクション '{collection_name}' を作成しました")

    collections = persistent_client.list_collections()
    print("現在のコレクション一覧:")
    for collection in collections:
        print(f"- {collection.name}")

    # ドキュメントをバッチで追加
    print(f"コレクション '{collection_name}' にデータをロードします")
    total_added = 0
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i + batch_size]
        vector_store.add_documents(documents=batch, embedding=embeddings)
        total_added += len(batch)
        print(f"バッチ {i//batch_size + 1} を追加しました（{len(batch)}件）")
    
    print(f"\n合計 {total_added} 件のドキュメントを追加しました")

except Exception as e:
    print(f"Chromaデータベースの操作中にエラーが発生しました: {str(e)}")


既存のコレクション 'mycollection_200' は存在しませんでした
コレクション 'mycollection_200' を作成しました
現在のコレクション一覧:
- mycollection_200
コレクション 'mycollection_200' にデータをロードします
バッチ 1 を追加しました（100件）
バッチ 2 を追加しました（100件）
バッチ 3 を追加しました（100件）
バッチ 4 を追加しました（100件）
バッチ 5 を追加しました（100件）
バッチ 6 を追加しました（100件）
バッチ 7 を追加しました（100件）
バッチ 8 を追加しました（100件）
バッチ 9 を追加しました（100件）
バッチ 10 を追加しました（100件）
バッチ 11 を追加しました（100件）
バッチ 12 を追加しました（100件）
バッチ 13 を追加しました（100件）
バッチ 14 を追加しました（100件）
バッチ 15 を追加しました（100件）
バッチ 16 を追加しました（100件）
バッチ 17 を追加しました（100件）
バッチ 18 を追加しました（100件）
バッチ 19 を追加しました（100件）
バッチ 20 を追加しました（100件）
バッチ 21 を追加しました（100件）
バッチ 22 を追加しました（100件）
バッチ 23 を追加しました（100件）
バッチ 24 を追加しました（100件）
バッチ 25 を追加しました（100件）
バッチ 26 を追加しました（100件）
バッチ 27 を追加しました（100件）
バッチ 28 を追加しました（100件）
バッチ 29 を追加しました（100件）
バッチ 30 を追加しました（100件）
バッチ 31 を追加しました（100件）
バッチ 32 を追加しました（100件）
バッチ 33 を追加しました（100件）
バッチ 34 を追加しました（75件）

合計 3375 件のドキュメントを追加しました


### ベクトル検索のテスト

In [8]:
result = vector_store.similarity_search(query, k=top_k)

In [9]:
for i, doc in enumerate(result[:top_k]):
    print(f"\n=== 検索結果 {i+1} ===")
    print(f"Content: {doc.page_content}")
    print(f"文字数: {len(doc.page_content)}")
    print(f"Source: {doc.metadata['source']}")


=== 検索結果 1 ===
Content: 敵対していたフェイトやヴィータなどの人物とも、幾度となく互いの信念をかけた戦いを繰り返した結果、最終的には分かり合うことが出来ており、彼女達と良き友となっている。使用デバイス(魔法の杖)はインテリジェントデバイス「レイジングハート」。ユーノから譲り受けたものだが、彼がこのデバイスを手に入れた経緯は不明。
文字数: 152
Source: data\Wikipedia-高町なのは.pdf

=== 検索結果 2 ===
Content: 彼がこの世界に来た理由。それは彼が発掘したロストロギア(異世界に存在した高度な魔法技術の遺産)「ジュエルシード」が散らばってしまったためであった。成り行きから事情を知ったなのはは、ユーノと共にジュエルシードを集め、封印する手伝いをすることになる。そして次第になのははユーノだけではなく、自分のために魔法の世界に関わっていく決意を固め、その秘めた力を開花させていく。
文字数: 182
Source: data\Wikipedia-魔法少女リリカルなのは.pdf

=== 検索結果 3 ===
Content: 新たなリーダーとなったひめなは静香に手を組むことを持ちかけるが拒否される。ひめなの指示のもと、羽根の神楽燦と遊狩ミユリがちはるの腕を切り落とし、石を強奪する。那由他は伯父である灯花の父に手はずを整えてもらい、同じく父の行方を探っている離婚した母と再会。父に関する調査資料を入手し、「湯国市」に手がかりがあるとわかって行動しようとする。
文字数: 167
Source: data\Wikipedia-マギアレコード_魔法少女まどか☆マギカ外伝.pdf

=== 検索結果 4 ===
Content: ストーリー自称平凡な小学3年生、高町なのはは助けを求める声に導かれ、不思議なフェレットが負傷し倒れているところを発見、保護する。その夜、再び声が響きフェレットを預かっている動物病院に向かうとそこで異形の怪物がフェレットと対峙する場面に出くわす。彼の正体は異世界ミッドチルダからやってきた少年ユーノ‧スクライアだった。彼がこの世界に来た理由。
文字数: 170
Source: data\Wikipedia-魔法少女リリカルなのは.pdf
